# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
# using Python 3.9.20
import pandas as pd 
import requests # this will be used to call the APIs

In [2]:
#choosing Miami as it has a decent number of entries at 167 rows
url = "http://api.citybik.es/v2/networks/citi-bike-miami"
# Make the API request
response = requests.get(url)
response

<Response [200]>

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [3]:
data = response.json()

Put your parsed results into a DataFrame.

In [4]:
#inspection of the json revealed two layers to dig through to get to our information
df = pd.DataFrame(data['network']['stations'])
extra_list = df['extra'] #create a new dataframe of just the extra column
df_extra = extra_list.apply(pd.Series) #turn the extras column into its own seperate table, not sure if we need it but don't want to throw it away yet
df.drop('extra', axis=1, inplace = True) # get rid of the extra column from the original dataframe




In [5]:
# on further study here we can see that the "name" column is a concatenation of "UID" and "address" from the extra column, so we will drop name and add extra back in as seperate columns
df['uid'] = df_extra['uid']
df['address'] = df_extra['address']
df.drop('name', axis=1, inplace = True)

In [6]:
#after inspection it is revealed "timestamp" column is currently a integer so lets convert that to a timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'])
#and we are going to rename "id" to "bs_id"(bike station id) so we dont get confused between stations
df.rename(columns = {'id':'bs_id'}, inplace = True)

In [7]:
#finally lets save this as a pickle file so it will preserve our formatting
df.to_pickle('../data/bike_dataframe.pkl')